In [1]:
from ue.uexp.dataprocessing.processor_binance import BinanceProcessor
from ue.uexp.dataprocessing.func import *
from ue.uexp.models.util import *
import tensorflow as tf
from tensorflow.keras import layers

import pandas as pd
import matplotlib.pyplot as plt


import warnings
warnings.filterwarnings('ignore')

In [2]:
### get data (set config params)
ticker_list = ["BTCUSDT"]
start_date = "2021-12-20"
end_date = "2021-12-31"
time_interval = "1m"

p = BinanceProcessor("binance")
df = p.download_data(ticker_list, start_date, end_date, time_interval)

## Preparing data for the N-BEATS algorithm using tf.data

In [3]:
WINDOW_SIZE = 30
HORIZON = 1

In [4]:
# 1 min data
df['time'] = pd.to_datetime(df['time'])
df.set_index('time', inplace=True)
df = df[['close']] #working with univariate model


In [5]:
df.head()

,close
time,
2021-12-20 00:00:00,46377.26
2021-12-20 00:01:00,46297.05
2021-12-20 00:02:00,46394.36
2021-12-20 00:03:00,46485.99
2021-12-20 00:04:00,46524.78


In [6]:
# Add windowed columns
df_windowed = df.copy()
for i in range(WINDOW_SIZE):
  df_windowed[f"Price+{i+1}"] = df_windowed["close"].shift(periods=i+1)
df_windowed.dropna().head()

,close,Price+1,Price+2,Price+3,Price+4,Price+5,Price+6,Price+7,Price+8,Price+9,...,Price+21,Price+22,Price+23,Price+24,Price+25,Price+26,Price+27,Price+28,Price+29,Price+30
time,,,,,,,,,,,,,,,,,,,,,
2021-12-20 00:30:00,46391.50,46388.68,46444.44,46487.01,46492.83,46511.57,46524.26,46586.24,46528.19,46550.00,...,46562.48,46565.28,46648.32,46541.70,46500.87,46524.78,46485.99,46394.36,46297.05,46377.26
2021-12-20 00:31:00,46396.01,46391.50,46388.68,46444.44,46487.01,46492.83,46511.57,46524.26,46586.24,46528.19,...,46547.50,46562.48,46565.28,46648.32,46541.70,46500.87,46524.78,46485.99,46394.36,46297.05
2021-12-20 00:32:00,46384.80,46396.01,46391.50,46388.68,46444.44,46487.01,46492.83,46511.57,46524.26,46586.24,...,46523.48,46547.50,46562.48,46565.28,46648.32,46541.70,46500.87,46524.78,46485.99,46394.36
2021-12-20 00:33:00,46426.47,46384.80,46396.01,46391.50,46388.68,46444.44,46487.01,46492.83,46511.57,46524.26,...,46540.44,46523.48,46547.50,46562.48,46565.28,46648.32,46541.70,46500.87,46524.78,46485.99
2021-12-20 00:34:00,46351.12,46426.47,46384.80,46396.01,46391.50,46388.68,46444.44,46487.01,46492.83,46511.57,...,46500.01,46540.44,46523.48,46547.50,46562.48,46565.28,46648.32,46541.70,46500.87,46524.78


In [7]:
# Make features and labels
X = df_windowed.dropna().drop("close", axis=1)
y = df_windowed.dropna()["close"]

# Make train and test sets
split_size = int(len(X) * 0.8)
X_train, y_train = X[:split_size], y[:split_size]
X_test, y_test = X[split_size:], y[split_size:]
len(X_train), len(y_train), len(X_test), len(y_test)

(6375, 6375, 1594, 1594)

### Now let's convert them into TensorFlow tf.data.Dataset's to ensure they run as fast as possible whilst training.

In [8]:
# 1. Turn train and test arrays into tensor Datasets
train_features_dataset = tf.data.Dataset.from_tensor_slices(X_train)
train_labels_dataset = tf.data.Dataset.from_tensor_slices(y_train)

test_features_dataset = tf.data.Dataset.from_tensor_slices(X_test)
test_labels_dataset = tf.data.Dataset.from_tensor_slices(y_test)

# 2. Combine features & labels
train_dataset = tf.data.Dataset.zip((train_features_dataset, train_labels_dataset))
test_dataset = tf.data.Dataset.zip((test_features_dataset, test_labels_dataset))

# 3. Batch and prefetch for optimal performance
BATCH_SIZE = 1024 # taken from Appendix D in N-BEATS paper
train_dataset = train_dataset.batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)
test_dataset = test_dataset.batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)

train_dataset, test_dataset

2022-02-03 16:58:36.194944: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-02-03 16:58:36.222140: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcusolver.so.11'; dlerror: libcusolver.so.11: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: :/usr/lib/nvidia:/home/m73/.mujoco/mujoco210/bin
2022-02-03 16:58:36.226307: W tensorflow/core/common_runtime/gpu/gpu_device.cc:1850] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...
2022-02-03 16:58:36.227310: I tensorflow/core/platform/cpu_feature_guard.cc:151] This Ten

(<PrefetchDataset shapes: ((None, 30), (None,)), types: (tf.float64, tf.float64)>,
 <PrefetchDataset shapes: ((None, 30), (None,)), types: (tf.float64, tf.float64)>)

## Setting up hyperparameters for N-BEATS algorithm
*using Figure 1 and Table 18/Appendix D of the N-BEATS paper.*

In [16]:
# Values from N-BEATS paper Figure 1 and Table 18/Appendix D
N_EPOCHS = 100 # called "Iterations" in Table 18
N_NEURONS = 512 # called "Width" in Table 18
N_LAYERS = 4
N_STACKS = 30

INPUT_SIZE = WINDOW_SIZE * HORIZON # called "Lookback" in Table 18
THETA_SIZE = INPUT_SIZE + HORIZON

INPUT_SIZE, THETA_SIZE

(30, 31)

## Notes: Building, compiling and fitting the N-BEATS algorithm
Okay, we've finally got all of the pieces of the puzzle ready for building and training the N-BEATS algorithm.

We'll do so by going through the following:

1. Setup an instance of the N-BEATS block layer using NBeatsBlock (this'll be the initial block used for the network, the rest will be created as part of stacks)
2. Create an input layer for the N-BEATS stack (we'll be using the Keras Functional API for this)
3. Make the initial backcast and forecasts for the model with the layer created in (1)
4. Use a for loop to create stacks of block layers
5. Use the NBeatsBlock class within the for loop created in (4) to create blocks which return backcasts and block-level forecasts
6. Create the double residual stacking using subtract and add layers
7. Put the model inputs and outputs together using tf.keras.Model()
8. Compile the model with MAE loss (the paper uses multiple losses but we'll use MAE to keep it inline with our other models) and Adam optimizer with default settings as per section 5.2 of N-BEATS paper)
9. Fit the N-BEATS model for 5000 epochs and since it's fitting for so many epochs, we'll use a couple of callbacks:
- tf.keras.callbacks.EarlyStopping() - stop the model from training if it doesn't improve validation loss for 200 epochs and restore the best performing weights using restore_best_weights=True (this'll prevent the model from training for loooongggggg period of time without improvement)
- tf.keras.callbacks.ReduceLROnPlateau() - if the model's validation loss doesn't improve for 100 epochs, reduce the learning rate by 10x to try and help it make incremental improvements (the smaller the learning rate, the smaller updates a model tries to make)

In [10]:
# 0. Defining the model

# Create NBeatsBlock custom layer 
class NBeatsBlock(tf.keras.layers.Layer):
  def __init__(self, # the constructor takes all the hyperparameters for the layer
               input_size: int,
               theta_size: int,
               horizon: int,
               n_neurons: int,
               n_layers: int,
               **kwargs): # the **kwargs argument takes care of all of the arguments for the parent class (input_shape, trainable, name)
    super().__init__(**kwargs)
    self.input_size = input_size
    self.theta_size = theta_size
    self.horizon = horizon
    self.n_neurons = n_neurons
    self.n_layers = n_layers

    # Block contains stack of 4 fully connected layers each has ReLU activation
    self.hidden = [tf.keras.layers.Dense(n_neurons, activation="relu") for _ in range(n_layers)]
    # Output of block is a theta layer with linear activation
    self.theta_layer = tf.keras.layers.Dense(theta_size, activation="linear", name="theta")

  def call(self, inputs): # the call method is what runs when the layer is called 
    x = inputs 
    for layer in self.hidden: # pass inputs through each hidden layer 
      x = layer(x)
    theta = self.theta_layer(x) 
    # Output the backcast and forecast from theta
    backcast, forecast = theta[:, :self.input_size], theta[:, -self.horizon:]
    return backcast, forecast

In [11]:
# 1. Setup N-BEATS Block layer
nbeats_block_layer = NBeatsBlock(input_size=INPUT_SIZE,
                                 theta_size=THETA_SIZE,
                                 horizon=HORIZON,
                                 n_neurons=N_NEURONS,
                                 n_layers=N_LAYERS,
                                 name="InitialBlock")


In [12]:
# 2. Create input to stacks
stack_input = layers.Input(shape=(INPUT_SIZE), name="stack_input")

In [13]:
# 3. Create initial backcast and forecast input (backwards predictions are referred to as residuals in the paper)
backcast, forecast = nbeats_block_layer(stack_input)
# Add in subtraction residual link, thank you to: https://github.com/mrdbourke/tensorflow-deep-learning/discussions/174 
residuals = layers.subtract([stack_input, backcast], name=f"subtract_00") 

In [15]:
# 4. Create stacks of blocks
for i, _ in enumerate(range(N_STACKS-1)): # first stack is already creted in (3)

  # 5. Use the NBeatsBlock to calculate the backcast as well as block forecast
  backcast, block_forecast = NBeatsBlock(
      input_size=INPUT_SIZE,
      theta_size=THETA_SIZE,
      horizon=HORIZON,
      n_neurons=N_NEURONS,
      n_layers=N_LAYERS,
      name=f"NBeatsBlock_{i}"
  )(residuals) # pass it in residuals (the backcast)

  # 6. Create the double residual stacking
  residuals = layers.subtract([residuals, backcast], name=f"subtract_{i}") 
  forecast = layers.add([forecast, block_forecast], name=f"add_{i}")


In [ ]:
# 7. Put the stack model together
model_7 = tf.keras.Model(inputs=stack_input, 
                         outputs=forecast, 
                         name="model_7_N-BEATS")

# 8. Compile with MAE loss and Adam optimizer
model_7.compile(loss="mae",
                optimizer=tf.keras.optimizers.Adam(0.001),
                metrics=["mae", "mse"])

# 9. Fit the model with EarlyStopping and ReduceLROnPlateau callbacks
model_7.fit(train_dataset,
            epochs=N_EPOCHS,
            validation_data=test_dataset,
            verbose=0, # prevent large amounts of training outputs
            # callbacks=[create_model_checkpoint(model_name=stack_model.name)] # saving model every epoch consumes far too much time
            callbacks=[tf.keras.callbacks.EarlyStopping(monitor="val_loss", patience=200, restore_best_weights=True),
                      tf.keras.callbacks.ReduceLROnPlateau(monitor="val_loss", patience=100, verbose=1)])